In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SolarCNNRegression(nn.Module):


    def __init__(self, input_channels=3, num_classes=1):
        super(SolarCNNRegression, self).__init__()


        self.features = nn.Sequential(

            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )


        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )


        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):

        features = self.features(x)


        irradiance = self.regressor(features)

        return irradiance


    def get_features(self, x):

        with torch.no_grad():
            features = self.features(x)
            return features.flatten(1)


class SolarCNNWithFeatureExtraction(SolarCNNRegression):


    def __init__(self, input_channels=3, feature_dim=512):
        super().__init__(input_channels, 1)
        self.feature_dim = feature_dim


        self.feature_projector = nn.Sequential(
            nn.Linear(512, feature_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2)
        )

    def forward(self, x, return_features=False):

        conv_features = self.features(x)
        flattened_features = conv_features.flatten(1)


        irradiance = self.regressor[-3:](self.regressor[:-3](flattened_features))

        if return_features:

            projected_features = self.feature_projector(flattened_features)
            return irradiance, projected_features

        return irradiance


class SolarCNN(SolarCNNRegression):

    pass

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import pandas as pd

class SolarIrradianceDataset(Dataset):


    def __init__(self, image_dir, irradiance_file, transform=None, target_size=(240, 320)):
        self.image_dir = image_dir
        self.transform = transform
        self.target_size = target_size


        if irradiance_file.endswith('.csv'):
            df = pd.read_csv(irradiance_file)

            self.irradiance_values = df.iloc[:, 1].values.astype(np.float32)
        else:
            self.irradiance_values = np.loadtxt(irradiance_file, delimiter=',')[:, 1].astype(np.float32)


        self.image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])


        min_length = min(len(self.image_files), len(self.irradiance_values))
        self.image_files = self.image_files[:min_length]
        self.irradiance_values = self.irradiance_values[:min_length]


        self.image_processor = IRImageProcessor(target_size=target_size)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):

        img_path = os.path.join(self.image_dir, self.image_files[idx])


        img = cv2.imread(img_path)
        if img is None:

            img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
            if img is not None:

                img = self.image_processor.process_single_image(img_path)
            else:
                raise ValueError(f"Could not load image: {img_path}")
        else:

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


        img_tensor = torch.tensor(img).permute(2, 0, 1).float() / 255.0


        if self.transform:
            img_tensor = self.transform(img_tensor)


        irradiance = torch.tensor(self.irradiance_values[idx], dtype=torch.float32)

        return img_tensor, irradiance


class SolarSequenceDataset(Dataset):


    def __init__(self, image_dir, irradiance_file, sequence_length=20, forecast_horizon=4, transform=None, target_size=(240, 320)):
        self.image_dir = image_dir
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon
        self.transform = transform
        self.target_size = target_size


        if irradiance_file.endswith('.csv'):
            df = pd.read_csv(irradiance_file)
            self.irradiance_values = df.iloc[:, 1].values.astype(np.float32)
        else:
            self.irradiance_values = np.loadtxt(irradiance_file, delimiter=',')[:, 1].astype(np.float32)


        self.image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])


        min_length = min(len(self.image_files), len(self.irradiance_values))
        self.image_files = self.image_files[:min_length]
        self.irradiance_values = self.irradiance_values[:min_length]


        self.valid_indices = list(range(len(self.image_files) - sequence_length - forecast_horizon + 1))


        self.image_processor = IRImageProcessor(target_size=target_size)

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        start_idx = self.valid_indices[idx]


        image_sequence = []
        for i in range(start_idx, start_idx + self.sequence_length):
            img_path = os.path.join(self.image_dir, self.image_files[i])


            img = cv2.imread(img_path)
            if img is None:
                img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
                if img is not None:
                    img = self.image_processor.process_single_image(img_path)
                else:
                    raise ValueError(f"Could not load image: {img_path}")
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


            img_tensor = torch.tensor(img).permute(2, 0, 1).float() / 255.0


            if self.transform:
                img_tensor = self.transform(img_tensor)

            image_sequence.append(img_tensor)


        image_sequence = torch.stack(image_sequence)


        historical_irradiance = torch.tensor(
            self.irradiance_values[start_idx:start_idx + self.sequence_length],
            dtype=torch.float32
        ).unsqueeze(-1)


        future_irradiance = torch.tensor(
            self.irradiance_values[start_idx + self.sequence_length: start_idx + self.sequence_length + self.forecast_horizon],
            dtype=torch.float32
        )

        return image_sequence, historical_irradiance, future_irradiance


class SolarTimeSeriesDataset(Dataset):


    def __init__(self, irradiance_file, sequence_length=20, forecast_horizon=4):
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon


        if irradiance_file.endswith('.csv'):
            df = pd.read_csv(irradiance_file)
            self.data = df.iloc[:, 1].values.astype(np.float32)
        else:
            self.data = np.loadtxt(irradiance_file, delimiter=',')[:, 1].astype(np.float32)


        self.valid_indices = list(range(len(self.data) - sequence_length - forecast_horizon + 1))

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        start_idx = self.valid_indices[idx]


        x = torch.tensor(
            self.data[start_idx:start_idx + self.sequence_length],
            dtype=torch.float32
        ).unsqueeze(-1)


        y = torch.tensor(
            self.data[start_idx + self.sequence_length: start_idx + self.sequence_length + self.forecast_horizon],
            dtype=torch.float32
        )

        return x, y


class GSIDataset(SolarIrradianceDataset):


    def __init__(self, image_dir, gsi_file):
        super().__init__(image_dir, gsi_file)

    def __getitem__(self, idx):
        img_tensor, irradiance = super().__getitem__(idx)
        return img_tensor, irradiance


class GSITimeSeriesDataset(SolarTimeSeriesDataset):


    def __init__(self, gsi_values, sequence_length=10):
        self.sequence_length = sequence_length
        self.data = gsi_values.astype(np.float32)
        self.valid_indices = list(range(len(self.data) - sequence_length))

    def __getitem__(self, idx):
        start_idx = self.valid_indices[idx]
        x = torch.tensor(
            self.data[start_idx:start_idx + self.sequence_length],
            dtype=torch.float32
        ).unsqueeze(-1)
        y = torch.tensor(self.data[start_idx + self.sequence_length], dtype=torch.float32)
        return x, y

In [ ]:
import torch
import torch.nn as nn

class SolarLSTMForecasting(nn.Module):


    def __init__(self, input_size=1, hidden_size=128, num_layers=2, output_size=4, dropout=0.2):
        super(SolarLSTMForecasting, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size


        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )


        lstm_output_size = hidden_size * 2

        self.fc_layers = nn.Sequential(
            nn.Linear(lstm_output_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(64, output_size)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        for name, param in self.lstm.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)


                n = param.size(0)
                start, end = n // 4, n // 2
                param.data[start:end].fill_(1.)

        for m in self.fc_layers:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):


        lstm_out, (hidden, cell) = self.lstm(x)


        last_output = lstm_out[:, -1, :]


        forecast = self.fc_layers(last_output)

        return forecast


class HybridCNNLSTM(nn.Module):


    def __init__(self, cnn_model=None, feature_dim=512, sequence_length=20, lstm_hidden_size=128, forecast_horizon=4):
        super(HybridCNNLSTM, self).__init__()

        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon


        if cnn_model is None:

            from __main__ import SolarCNNWithFeatureExtraction
            self.cnn = SolarCNNWithFeatureExtraction(feature_dim=feature_dim)
        else:
            self.cnn = cnn_model


        self.lstm = SolarLSTMForecasting(
            input_size=1,
            hidden_size=lstm_hidden_size,
            output_size=forecast_horizon
        )


        self.freeze_cnn = False

    def set_cnn_trainable(self, trainable=True):

        for param in self.cnn.parameters():
            param.requires_grad = trainable
        self.freeze_cnn = not trainable

    def forward(self, image_sequence):






        batch_size, seq_len, channels, height, width = image_sequence.shape


        images_flat = image_sequence.view(-1, channels, height, width)


        nowcasts = self.cnn(images_flat)
        nowcasts = nowcasts.view(batch_size, seq_len, 1)


        forecasts = self.lstm(nowcasts)

        return nowcasts, forecasts

    def predict_from_sequence(self, image_sequence):

        self.eval()
        with torch.no_grad():
            nowcasts, forecasts = self.forward(image_sequence)
        return nowcasts, forecasts


class SolarLSTM(nn.Module):


    def __init__(self, input_size=1, hidden_size=128, output_size=1):
        super(SolarLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import json
import cv2
import pandas as pd
from preprocess import IRImageProcessor

class SolarIrradianceDataset(Dataset):


    def __init__(self, image_dir, irradiance_file, transform=None, target_size=(240, 320)):
        self.image_dir = image_dir
        self.transform = transform
        self.target_size = target_size


        if irradiance_file.endswith('.csv'):
            df = pd.read_csv(irradiance_file)

            self.irradiance_values = df.iloc[:, 1].values.astype(np.float32)
        else:
            self.irradiance_values = np.loadtxt(irradiance_file, delimiter=',')[:, 1].astype(np.float32)


        self.image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])


        min_length = min(len(self.image_files), len(self.irradiance_values))
        self.image_files = self.image_files[:min_length]
        self.irradiance_values = self.irradiance_values[:min_length]


        self.image_processor = IRImageProcessor(target_size=target_size)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):

        img_path = os.path.join(self.image_dir, self.image_files[idx])


        img = cv2.imread(img_path)
        if img is None:

            img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
            if img is not None:

                img = self.image_processor.process_single_image(img_path)
            else:
                raise ValueError(f"Could not load image: {img_path}")
        else:

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


        img_tensor = torch.tensor(img).permute(2, 0, 1).float() / 255.0


        if self.transform:
            img_tensor = self.transform(img_tensor)


        irradiance = torch.tensor(self.irradiance_values[idx], dtype=torch.float32)

        return img_tensor, irradiance


class SolarSequenceDataset(Dataset):


    def __init__(self, image_paths, irradiance_values, sequence_length=20, forecast_horizon=4, transform=None, target_size=(240, 320)):
        self.image_paths = image_paths
        self.irradiance_values = irradiance_values
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon
        self.transform = transform
        self.target_size = target_size


        self.valid_indices = list(range(len(self.image_paths) - sequence_length - forecast_horizon + 1))


        self.image_processor = IRImageProcessor(target_size=target_size)


    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        start_idx = self.valid_indices[idx]


        image_sequence = []
        for i in range(start_idx, start_idx + self.sequence_length):
            img_path = self.image_paths[i]


            img = cv2.imread(img_path)
            if img is None:
                img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
                if img is not None:
                    img = self.image_processor.process_single_image(img_path)
                else:
                    raise ValueError(f"Could not load image: {img_path}")
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


            img_tensor = torch.tensor(img).permute(2, 0, 1).float() / 255.0


            if self.transform:
                img_tensor = self.transform(img_tensor)

            image_sequence.append(img_tensor)


        image_sequence = torch.stack(image_sequence)


        historical_irradiance = torch.tensor(
            self.irradiance_values[start_idx:start_idx + self.sequence_length],
            dtype=torch.float32
        ).unsqueeze(-1)


        future_irradiance = torch.tensor(
            self.irradiance_values[start_idx + self.sequence_length: start_idx + self.sequence_length + self.forecast_horizon],
            dtype=torch.float32
        )

        return image_sequence, historical_irradiance, future_irradiance


class SolarTimeSeriesDataset(Dataset):


    def __init__(self, irradiance_file, sequence_length=20, forecast_horizon=4):
        self.sequence_length = sequence_length
        self.forecast_horizon = forecast_horizon


        if irradiance_file.endswith('.csv'):
            df = pd.read_csv(irradiance_file)
            self.data = df.iloc[:, 1].values.astype(np.float32)
        else:
            self.data = np.loadtxt(irradiance_file, delimiter=',')[:, 1].astype(np.float32)


        self.valid_indices = list(range(len(self.data) - sequence_length - forecast_horizon + 1))

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        start_idx = self.valid_indices[idx]


        x = torch.tensor(
            self.data[start_idx:start_idx + self.sequence_length],
            dtype=torch.float32
        ).unsqueeze(-1)


        y = torch.tensor(
            self.data[start_idx + self.sequence_length: start_idx + self.sequence_length + self.forecast_horizon],
            dtype=torch.float32
        )

        return x, y


class GSIDataset(SolarIrradianceDataset):


    def __init__(self, image_dir, gsi_file):
        super().__init__(image_dir, gsi_file)

    def __getitem__(self, idx):
        img_tensor, irradiance = super().__getitem__(idx)
        return img_tensor, irradiance


class GSITimeSeriesDataset(SolarTimeSeriesDataset):


    def __init__(self, gsi_values, sequence_length=10):
        self.sequence_length = sequence_length
        self.data = gsi_values.astype(np.float32)
        self.valid_indices = list(range(len(self.data) - sequence_length))

    def __getitem__(self, idx):
        start_idx = self.valid_indices[idx]
        x = torch.tensor(
            self.data[start_idx:start_idx + self.sequence_length],
            dtype=torch.float32
        ).unsqueeze(-1)
        y = torch.tensor(self.data[start_idx + self.sequence_length], dtype=torch.float32)
        return x, y


def get_multi_day_sequence_dataset(image_dirs, irradiance_files, sequence_length, forecast_horizon):
    image_paths = []
    irradiance_values = []

    for img_dir, irr_file in zip(image_dirs, irradiance_files):

        if irr_file.endswith('.csv'):
            df = pd.read_csv(irr_file)
            values = df.iloc[:, 1].values.astype(np.float32)
        else:
            values = np.loadtxt(irr_file, delimiter=',')[:, 1].astype(np.float32)

        files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        files = files[:len(values)]

        image_paths.extend([os.path.join(img_dir, f) for f in files])
        irradiance_values.extend(values[:len(files)])

    return SolarSequenceDataset(image_paths, irradiance_values, sequence_length, forecast_horizon)


class HybridTrainer:


    def __init__(self, pretrained_cnn_path=None, config=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")


        self.config = {
            'sequence_length': 20,
            'forecast_horizon': 4,
            'learning_rate': 1e-4,
            'batch_size': 16,
            'num_epochs': 30,
            'lstm_hidden_size': 128,
            'feature_dim': 512,
            'weight_decay': 1e-4,
            'scheduler_patience': 8,
            'early_stopping_patience': 12,
            'freeze_cnn': True,
            'save_dir': 'models',
            'log_dir': 'logs'
        }

        if config:
            self.config.update(config)


        os.makedirs(self.config['save_dir'], exist_ok=True)
        os.makedirs(self.config['log_dir'], exist_ok=True)


        cnn_model = None
        if pretrained_cnn_path:
            print(f"Loading pretrained CNN from {pretrained_cnn_path}")
            cnn_model = SolarCNNWithFeatureExtraction()
            checkpoint = torch.load(pretrained_cnn_cnn_model.state_dict(), map_location=self.device)
            cnn_model.load_state_dict(checkpoint['model_state_dict'])

        self.model = HybridCNNLSTM(
            cnn_model=cnn_model,
            feature_dim=self.config['feature_dim'],
            sequence_length=self.config['sequence_length'],
            lstm_hidden_size=self.config['lstm_hidden_size'],
            forecast_horizon=self.config['forecast_horizon']
        ).to(self.device)


        self.model.set_cnn_trainable(not self.config['freeze_cnn'])


        if self.config['freeze_cnn']:

            lstm_params = list(self.model.lstm.parameters())
            print(f"Training LSTM only ({sum(p.numel() for p in lstm_params):,} parameters)")
        else:

            lstm_params = self.model.parameters()
            print(f"Training full hybrid model ({sum(p.numel() for p in self.model.parameters()):,} parameters)")

        self.optimizer = optim.Adam(
            lstm_params,
            lr=self.config['learning_rate'],
            weight_decay=self.config['weight_decay']
        )

        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        self.optimizer,
        mode='min',
        patience=self.config['scheduler_patience'],
        factor=0.5
        )


        self.mse_loss = nn.MSELoss()
        self.mae_loss = nn.L1Loss()


        self.train_losses = []
        self.val_losses = []
        self.best_val_loss = float('inf')
        self.patience_counter = 0

    def train_epoch(self, train_loader):

        self.model.train()
        total_nowcast_loss = 0.0
        total_forecast_loss = 0.0
        total_loss = 0.0
        num_batches = 0

        with tqdm(train_loader, desc="Training Hybrid") as pbar:
            for batch_idx, (image_sequences, historical_irradiance, future_irradiance) in enumerate(pbar):
                image_sequences = image_sequences.to(self.device)
                historical_irradiance = historical_irradiance.to(self.device)
                future_irradiance = future_irradiance.to(self.device)

                self.optimizer.zero_grad()


                nowcasts, forecasts = self.model(image_sequences)


                nowcast_loss = self.mse_loss(nowcasts.squeeze(-1), historical_irradiance.squeeze(-1))
                forecast_loss = self.mse_loss(forecasts, future_irradiance)


                total_batch_loss = 0.3 * nowcast_loss + 0.7 * forecast_loss

                total_batch_loss.backward()


                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

                self.optimizer.step()

                total_nowcast_loss += nowcast_loss.item()
                total_forecast_loss += forecast_loss.item()
                total_loss += total_batch_loss.item()
                num_batches += 1


                pbar.set_postfix({
                    'Nowcast': f'{nowcast_loss.item():.4f}',
                    'Forecast': f'{forecast_loss.item():.4f}',
                    'Total': f'{total_batch_loss.item():.4f}'
                })

        return (total_nowcast_loss / num_batches,
                total_forecast_loss / num_batches,
                total_loss / num_batches)

    def validate_epoch(self, val_loader):

        self.model.eval()
        total_nowcast_loss = 0.0
        total_forecast_loss = 0.0
        all_nowcast_pred = []
        all_nowcast_target = []
        all_forecast_pred = []
        all_forecast_target = []

        with torch.no_grad():
            with tqdm(val_loader, desc="Validation") as pbar:
                for image_sequences, historical_irradiance, future_irradiance in pbar:
                    image_sequences = image_sequences.to(self.device)
                    historical_irradiance = historical_irradiance.to(self.device)
                    future_irradiance = future_irradiance.to(self.device)

                    nowcasts, forecasts = self.model(image_sequences)

                    nowcast_loss = self.mse_loss(nowcasts.squeeze(-1), historical_irradiance.squeeze(-1))
                    forecast_loss = self.mse_loss(forecasts, future_irradiance)

                    total_nowcast_loss += nowcast_loss.item()
                    total_forecast_loss += forecast_loss.item()


                    all_nowcast_pred.append(nowcasts.squeeze(-1).cpu().numpy())
                    all_nowcast_target.append(historical_irradiance.squeeze(-1).cpu().numpy())
                    all_forecast_pred.append(forecasts.cpu().numpy())
                    all_forecast_target.append(future_irradiance.cpu().numpy())

                    pbar.set_postfix({
                        'Nowcast': f'{nowcast_loss.item():.4f}',
                        'Forecast': f'{forecast_loss.item():.4f}'
                    })

        avg_nowcast_loss = total_nowcast_loss / len(val_loader)
        avg_forecast_loss = total_forecast_loss / len(val_loader)


        nowcast_pred = np.concatenate(all_nowcast_pred, axis=0)
        nowcast_target = np.concatenate(all_nowcast_target, axis=0)
        forecast_pred = np.concatenate(all_forecast_pred, axis=0)
        forecast_target = np.concatenate(all_forecast_target, axis=0)


        nowcast_rmse = np.sqrt(mean_squared_error(nowcast_target.flatten(), nowcast_pred.flatten()))


        forecast_rmse = np.sqrt(mean_squared_error(forecast_target.flatten(), forecast_pred.flatten()))


        forecast_rmse_per_step = []
        for step in range(self.config['forecast_horizon']):
            step_rmse = np.sqrt(mean_squared_error(forecast_target[:, step], forecast_pred[:, step]))
            forecast_rmse_per_step.append(step_rmse)

        return (avg_nowcast_loss, avg_forecast_loss, nowcast_rmse, forecast_rmse, forecast_rmse_per_step)

    def train(self, train_loader, val_loader=None):

        print(f"Starting hybrid CNN-LSTM training for {self.config['num_epochs']} epochs...")
        print(f"CNN frozen: {self.config['freeze_cnn']}")

        for epoch in range(self.config['num_epochs']):
            print(f"\nEpoch {epoch+1}/{self.config['num_epochs']}")


            train_nowcast_loss, train_forecast_loss, train_total_loss = self.train_epoch(train_loader)
            self.train_losses.append(train_total_loss)

            print(f"Train - Nowcast: {train_nowcast_loss:.4f}, Forecast: {train_forecast_loss:.4f}, Total: {train_total_loss:.4f}")


            if val_loader is not None:
                (val_nowcast_loss, val_forecast_loss, nowcast_rmse, forecast_rmse, forecast_rmse_per_step) = self.validate_epoch(val_loader)

                val_total_loss = 0.3 * val_nowcast_loss + 0.7 * val_forecast_loss
                self.val_losses.append(val_total_loss)

                print(f"Val - Nowcast RMSE: {nowcast_rmse:.2f} W/m², Forecast RMSE: {forecast_rmse:.2f} W/m²")
                print(f"Forecast RMSE per step: {[f'{r:.2f}' for r in forecast_rmse_per_step]}")


                self.scheduler.step(val_total_loss)


                if val_total_loss < self.best_val_loss:
                    self.best_val_loss = val_total_loss
                    self.patience_counter = 0


                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': self.model.state_dict(),
                        'optimizer_state_dict': self.optimizer.state_dict(),
                        'val_loss': val_total_loss,
                        'nowcast_rmse': nowcast_rmse,
                        'forecast_rmse': forecast_rmse,
                        'config': self.config
                    }, os.path.join(self.config['save_dir'], 'best_hybrid_model.pth'))

                    print(f"New best model saved! Forecast RMSE: {forecast_rmse:.2f} W/m²")
                else:
                    self.patience_counter += 1

                if self.patience_counter >= self.config['early_stopping_patience']:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break


        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'train_losses': self.train_losses,
            'val_losses': self.val_losses
        }, os.path.join(self.config['save_dir'], 'final_hybrid_cnn_with_features_model.pth'))


        history = {
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'config': self.config
        }

        with open(os.path.join(self.config['log_dir'], 'hybrid_training_history.json'), 'w') as f:
            json.dump(history, f, indent=2)

        print("Hybrid CNN-LSTM training completed!")
        return self.train_losses, self.val_losses


def train_hybrid_model():


    config = {
        'sequence_length': 20,
        'forecast_horizon': 4,
        'learning_rate': 1e-4,
        'batch_size': 128,
        'num_epochs': 50,
        'freeze_cnn': True,

        'image_dirs': [
            '/content/drive/MyDrive/data/processed/2019_01_15',
            '/content/drive/MyDrive/data/processed/2019_01_16',
            '/content/drive/MyDrive/data/processed/2019_01_17',
            '/content/drive/MyDrive/data/processed/2019_01_18'


        ],
        'irradiance_files': [
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_15/pyranometer/2019_01_15.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_16/pyranometer/2019_01_16.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_17/pyranometer/2019_01_17.csv',
            '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_18/pyranometer/2019_01_18.csv'
        ],
        'pretrained_cnn_path': '/content/drive/MyDrive/models(with_feature_cnn)/best_cnn_with_features_model.pth'
    }


    print("Loading multi-day sequence dataset...")
    dataset = get_multi_day_sequence_dataset(
        image_dirs=config['image_dirs'],
        irradiance_files=config['irradiance_files'],
        sequence_length=config['sequence_length'],
        forecast_horizon=config['forecast_horizon']
    )


    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )


    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=2
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=2
    )

    print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")


    trainer = HybridTrainer(
        pretrained_cnn_path=config.get('pretrained_cnn_path'),
        config=config
    )


    train_losses, val_losses = trainer.train(train_loader, val_loader)

    print("Hybrid CNN-LSTM training completed!")


if __name__ == '__main__':
    train_hybrid_model()

Loading multi-day sequence dataset...
Train samples: 5381, Val samples: 1346
Using device: cpu
Loading pretrained CNN from /content/drive/MyDrive/models(with_feature_cnn)/best_cnn_with_features_model.pth
Training LSTM only (570,820 parameters)
Starting hybrid CNN-LSTM training for 50 epochs...
CNN frozen: True

Epoch 1/50


Training Hybrid:   0%|          | 0/43 [00:00<?, ?it/s]